In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [8]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [9]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [11]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [13]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [14]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


2023-02-13 23:20:01.854491: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [15]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [17]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 1.7852017e-03  6.1457808e-04 -2.4946202e-03 ...  5.4947417e-03
    5.0941012e-03 -8.8060347e-05]
  [ 1.1672266e-05 -1.4731460e-03  1.0857894e-03 ... -9.9544693e-04
    8.1720622e-03  1.7027390e-03]
  [-3.3933839e-03 -1.9652062e-04 -1.2926784e-03 ...  3.5510403e-03
    4.9258387e-03 -1.5528948e-03]
  ...
  [-1.0975113e-03  2.8378682e-03 -8.9973118e-03 ...  1.1726836e-03
    3.8203404e-03  1.0832231e-03]
  [ 5.8180867e-03 -5.0577195e-03 -7.8686960e-03 ...  1.1396753e-03
    4.5869416e-03 -5.7543744e-04]
  [ 1.1292380e-02 -2.7070572e-03 -8.7875417e-03 ...  6.9863014e-03
    4.0852749e-03 -2.2873515e-03]]

 [[ 1.7852017e-03  6.1457808e-04 -2.4946202e-03 ...  5.4947417e-03
    5.0941012e-03 -8.8060347e-05]
  [ 7.0557441e-03  3.5182727e-03 -2.7715107e-03 ...  9.4597945e-03
    2.7606145e-03 -2.6310226e-03]
  [ 1.6668388e-03  9.7116167e-03  1.0372318e-03 ...  5.7866001e-03
    3.0862524e-03 -5.8719906e-04]
  ...
  [ 1.1847350e-02 -2.0987585e-03 -2.2501824e-03 ...  5.4496585e

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, 
#where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 1.7852017e-03  6.1457808e-04 -2.4946202e-03 ...  5.4947417e-03
   5.0941012e-03 -8.8060347e-05]
 [ 1.1672266e-05 -1.4731460e-03  1.0857894e-03 ... -9.9544693e-04
   8.1720622e-03  1.7027390e-03]
 [-3.3933839e-03 -1.9652062e-04 -1.2926784e-03 ...  3.5510403e-03
   4.9258387e-03 -1.5528948e-03]
 ...
 [-1.0975113e-03  2.8378682e-03 -8.9973118e-03 ...  1.1726836e-03
   3.8203404e-03  1.0832231e-03]
 [ 5.8180867e-03 -5.0577195e-03 -7.8686960e-03 ...  1.1396753e-03
   4.5869416e-03 -5.7543744e-04]
 [ 1.1292380e-02 -2.7070572e-03 -8.7875417e-03 ...  6.9863014e-03
   4.0852749e-03 -2.2873515e-03]], shape=(100, 65), dtype=float32)


In [20]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 1.7852017e-03  6.1457808e-04 -2.4946202e-03  1.8026571e-03
  2.8827037e-03 -2.2151083e-04 -3.6322160e-04  2.0673471e-03
  9.2511596e-03  4.5862771e-04  4.7195391e-03  8.7708505e-03
 -8.5869133e-03 -1.6229483e-03 -5.3940774e-03  5.3539989e-05
  3.5815658e-03  2.8615296e-03 -4.5555220e-03 -1.6005746e-03
  4.1341810e-03  1.5032027e-03  2.1295692e-04  1.5932377e-03
  2.7101119e-03 -1.6926206e-03  1.1901625e-03 -1.1284995e-03
  4.2640842e-03  4.4058114e-03  2.5339003e-03  3.5841279e-03
  5.3705066e-05 -6.5710291e-04  1.8018604e-03 -5.3375983e-03
 -9.3271001e-04 -6.6649378e-04  1.0875711e-03  2.2319076e-04
 -6.6640868e-04  3.0010531e-03 -4.7944738e-03 -2.0600269e-03
 -3.9278162e-03 -3.0950962e-03 -2.4123751e-03 -1.3126438e-03
 -4.3254482e-04  4.3852152e-03 -7.6125921e-03  3.1936341e-03
 -6.2210076e-03 -2.0423601e-03 -8.4540644e-04  2.3380013e-03
  9.4754878e-04 -7.3730975e-04  7.3731486e-03 -3.0999135e-03
 -1.4885095e-03  2.6145708e-03  5.4947417e-03  5.0941012e-03
 -8.806034

In [22]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

".P-b;MrUF'!qa!LErTRQ&yV&yls:Yz,M\nIMftcF3s!Wph,XmXX J&kUoz;:Z$dn3fjG-IfcYHlwUI.x Z,MQDmr$-CbdKUV-?QwW"

In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 192s 1s/step - loss: 2.5999
Epoch 2/50
172/172 [==============================] - 220s 1s/step - loss: 1.8894
Epoch 3/50
172/172 [==============================] - 214s 1s/step - loss: 1.6387
Epoch 4/50
172/172 [==============================] - 253s 1s/step - loss: 1.5057
Epoch 5/50
172/172 [==============================] - 268s 2s/step - loss: 1.4226
Epoch 6/50
172/172 [==============================] - 280s 2s/step - loss: 1.3655
Epoch 7/50
172/172 [==============================] - 282s 2s/step - loss: 1.3208
Epoch 8/50
  4/172 [..............................] - ETA: 4:01 - loss: 1.2732

In [32]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [33]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [34]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
    num_generate = 800

  # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
    text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
    temperature = 1.0

  # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      # remove the batch dimension
    
        predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [6]:
inp = input("Enter the starting string ")
print(generate_text(model, inp))

Enter the starting string Hello


NameError: name 'generate_text' is not defined